In [1]:
using PorousMaterials, CSV, LightGraphs
using PyCall
using LinearAlgebra
scipy = pyimport("scipy.spatial")

@eval PorousMaterials PATH_TO_CRYSTALS = joinpath(pwd(), "xtals")

"/home/cokes/mpn_charges/build_graphs/xtals"

In [2]:
function cordero_covalent_atomic_radii()
    df = CSV.read("covalent_radii.csv", comment="#")
    atom_to_radius = Dict{Symbol, Float64}()
    for atom in eachrow(df)
        atom_to_radius[Symbol(atom[:atom])] = atom[:covalent_radius_A]
    end
    return atom_to_radius
end

const radii = cordero_covalent_atomic_radii()

Dict{Symbol,Float64} with 105 entries:
  :Ra          => 2.21
  :Cl          => 1.02
  :Al          => 1.21
  :Be          => 0.96
  :Re          => 1.51
  :Cr          => 1.39
  :Na          => 1.66
  :Csp3        => 0.76
  :Sb          => 1.39
  :Kr          => 1.16
  :Ni          => 1.24
  :S           => 1.05
  :Fe_low_spin => 1.32
  :Ru          => 1.46
  :Tl          => 1.45
  :Tm          => 1.9
  :W           => 1.62
  :O           => 0.66
  :Nd          => 2.01
  :Tb          => 1.94
  :Th          => 2.06
  :Zr          => 1.75
  :F           => 0.57
  :Co          => 1.5
  :Fr          => 2.6
  ⋮            => ⋮

In [3]:
framework = Framework("KAXQIL_clean.cif")
framework = replicate(framework, (2, 2, 2))

Name: KAXQIL_clean.cif
Bravais unit cell of a crystal.
	Unit cell angles α = 100.355998 deg. β = 90.000000 deg. γ = 90.000000 deg.
	Unit cell dimensions a = 11.134600 Å. b = 23.642800 Å, c = 45.520600 Å
	Volume of unit cell: 11788.227353 Å³

Number of atoms = 960
Number of charges = 960
Chemical formula: Dict(:H => 8,:S => 1,:Ca => 1,:O => 6,:C => 14)
Space Group: P1
Symmetry Operations:
	'x, y, z'


In [8]:
function adjacency_matrix(framework::Framework, apply_pbc::Bool)
    A = zeros(framework.atoms.n_atoms, framework.atoms.n_atoms)
    for i = 1:framework.atoms.n_atoms
        for j = (i+1):framework.atoms.n_atoms
            A[i, j] = distance(framework, i, j, apply_pbc)
            A[j, i] = A[i, j] # symmetry
        end
    end
    return A
end

function neighborhood(framework::Framework, i::Int, r::Float64, am::Array{Float64, 2})
    # get indices of atoms within a distance r of atom i
    ids_neighbors = findall((am[:, i] .> 0.0) .& (am[:, i] .< r))
    
    # rs is the list of distance of neighbors from atom i
    rs = [am[i, id_n] for id_n in ids_neighbors]
    @assert all(rs .< r)
    
    # xs is a list of Cartesian coords of the neighborhood
    #   coords of atom i are subtracted off
    #   first entry is coords of atom i, the center, the zero vector
    #   remaining entries are neighbors
    # this list is useful to pass to Voronoi for getting Voronoi faces
    #    of the neighborhood.
    xs = [[0.0, 0.0, 0.0]] # this way atom zero is itself
    for j in ids_neighbors
        # subtract off atom i, apply nearest image
        xf = framework.atoms.xf[:, j] - framework.atoms.xf[:, i]
        nearest_image!(xf)
        x = framework.box.f_to_c * xf
        push!(xs, x)
#         @assert isapprox(norm(x), am[i, j])
    end

    return ids_neighbors, xs, rs
end

function shared_voronoi_faces(ids_neighbors::Array{Int, 1}, 
                              xs::Array{Array{Float64,1},1})
    @assert(length(ids_neighbors) == length(xs) - 1)
    
    voro = scipy.Voronoi(xs)
    rps = voro.ridge_points # connections with atom zero are connections with atom i.
    ids_shared_faces = Int[] # corresponds to xs, not to atoms of framework
    for k = 1:size(rps)[1]
        if rps[k, 1] == 0 # a shared face with atom i
            push!(ids_shared_faces, rps[k, 2])
        end
        if rps[k, 2] == 0 # a shared face with atom i
            push!(ids_shared_faces, rps[k, 1])
        end
    end
    return ids_neighbors[ids_shared_faces]
end

am = adjacency_matrix(framework, true)

function bonded_atoms(framework::Framework, i::Int, am::Array{Float64, 2})
    species_i = framework.atoms.species[i]
    
    ids_neighbors, xs, rs = neighborhood(framework, i, 6.0, am)
    
    ids_shared_voro_faces = shared_voronoi_faces(ids_neighbors, xs)
    
    ids_bonded = Int[]
    for j in ids_shared_voro_faces
        species_j = framework.atoms.species[j]
        # sum of covalent radii
        radii_sum = radii[species_j] + radii[species_i]
        if am[i, j] < radii_sum + 0.25
            push!(ids_bonded, j)
        end
    end
    return ids_bonded
end

function bonds!(framework::Framework, apply_pbc::Bool)
    am = adjacency_matrix(framework, apply_pbc)
    
    for i = 1:framework.atoms.n_atoms
        for j in bonded_atoms(framework, i, am)
            add_edge!(framework.bonds, i, j)
        end
    end
end
    
bonds!(framework, true)

In [9]:
write_bond_information(framework)
write_xyz(framework)

Saving bond information for framework KAXQIL_clean.cif to /home/cokes/mpn_charges/build_graphs/KAXQIL_clean.cif_bonds.vtk.
